In [205]:
import nltk
import numpy.matlib
from nltk.grammar import *
from pprint import pprint

# Loading the Corpus and Grammars

In [206]:
# Loads the ATIS CNF grammar and the sentences. If the sentences file is not available,
# automatically downloads it and loads it

try:
    grammar = nltk.data.load('atis/atis-grammar-cnf.cfg')
    s = nltk.data.load('grammars/large_grammars/atis_sentences.txt')
    t = nltk.parse.util.extract_test_sentences(s)
    
except LookupError:
    nltk.download('large_grammars')
    
    grammar = nltk.data.load('atis/atis-grammar-cnf.cfg')
    s = nltk.data.load('grammars/large_grammars/atis_sentences.txt')
    t = nltk.parse.util.extract_test_sentences(s)

In [ ]:
with open('atis/atis-grammar-cnf.cfg','r') as string_cfg:
    grammar1 = nltk.grammar.CFG.fromstring(string_cfg.read())
type(grammar1)

## Test Grammar

In [ ]:
test_grammar_string = '''
S -> NP VP
NP -> Det N
NP -> NP PP
PP -> P NP
VP -> V NP
VP -> VP PP

NP -> 'I'
N -> 'elephant'
N -> 'pajamas'
V -> 'shot'
P -> 'in'
Det -> 'an'
Det -> 'my'
'''

test_grammar = nltk.grammar.CFG.fromstring(test_grammar_string)
test_grammar.productions()

In [ ]:
class CKY:
    def __init__(self,sentence, grammar):
        '''Class created for the CKY algorithm. It automatically recognizes the sentence
        at first, if it is in the grammar we can generate the parse tree
        :param sentence: sentence to be parser
        :type sentence: str
        :param grammar: grammar of the language
        :type NLTK.grammar.CFG
        '''
        self.grammar = grammar
        self.sent_list = sentence.split()
        n = len(self.sent_list)
        
        self.chart = [[None for i in range(n)] for j in range(n)]
        
        
        for i in range(1, n+1):
            # Adds all possible terminal nodes according to the grammar
            self.chart[n-i][i-1] = [rule.lhs()
                            for rule in self.grammar.productions(rhs=self.sent_list[i-1])]
            
        for b in range(2, n+1):
            for i in range(0, n-b+1):
                list_of_As = []
                for k in range(1, b):
                    B = self.chart[n-i-k][i]
                    C = self.chart[n-b-i][i + k]
                    if B and C:
                        # If B and C exist then compare them. 
                        # Saves checking every possible k point
                        list_of_As.append(CKY.check_grammar(
                            self.grammar,B,C))
                    else:
                        continue       
                # Filters the list of None objects
                self.chart[n-b-i][i] = list(filter(None,list_of_As))

                
    
    def check_grammar(gram, B,C):
        '''Function created to check if a production with two Non terminals B and C,
        exist in a language
        :param gram: grammar of the language 
        :type gram: nltk.grammar.CFG
        :param B,C: list of all possible Nonterminal objects for the righ corner(B) and
                    for the left corner(C) of the production rule
        :type: list
        :returns: a Non terminal corresponding to the left hand side of the found rule
        :return type: nltk.grammar.CFG.Nonterminal
        '''
        for element_B in B:
            # for every production with element_B on the left side of the
            # righ hand side, check if there is a production with element_C
            # on the right side of right hand side
            
            for prod in gram.productions(rhs=element_B):
                for element_C in C:
                    # the method .rhs() from the NLTK object Production consists
                    # of a tuple with two Nonterminal objects. Therefore the comparison
                    # works.
                    if prod.rhs() == (element_B, element_C):
                        return(prod.lhs())

    
    def print_chart(self):
        pprint(self.chart)
        


In [ ]:
racog = CKY('I shot an elephant in my pajamas', test_grammar)
racog.print_chart()